# STAR PERFORMERS

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
import xmltodict
from pprint import pprint
import os
import csv
from api_keys import z_keys

In [2]:
ny18_df = pd.read_csv("Files/2018NYSales.csv")

In [3]:
len(ny18_df)

16369

In [5]:
ny18_df = ny18_df.rename(columns={' SALE PRICE ': 'SALE PRICE'})

In [6]:
ny18_df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [ ]:
ny18_df.describe()

In [7]:
ny18_df["ADDRESS"] = ny18_df["ADDRESS"].str.strip()
ny18_df['ADDRESS'] = ny18_df['ADDRESS'].str.split(' ').apply(lambda x: '+'.join(x))
ny18_df.count()

BOROUGH                           16369
NEIGHBORHOOD                      16369
BUILDING CLASS CATEGORY           16369
TAX CLASS AT PRESENT              16369
BLOCK                             16369
LOT                               16369
EASE-MENT                         16369
BUILDING CLASS AT PRESENT         16369
ADDRESS                           16369
APARTMENT NUMBER                  16369
ZIP CODE                          16369
RESIDENTIAL UNITS                 16369
COMMERCIAL UNITS                  16369
TOTAL UNITS                       16369
LAND SQUARE FEET                  16369
GROSS SQUARE FEET                 16369
YEAR BUILT                        16369
TAX CLASS AT TIME OF SALE         16369
BUILDING CLASS AT TIME OF SALE    16369
SALE PRICE                        16369
SALE DATE                         16369
dtype: int64

In [8]:
ny18_df["SALE PRICE"] = ny18_df.loc[:,"SALE PRICE"].replace('[\$,]', '', regex=True)
ny18_df["SALE PRICE"] = ny18_df.loc[:,"SALE PRICE"].replace('[-,]', '0', regex=True)

In [9]:
# Removing home prices less than 100K to avoid a bias after preliminary research on what these lower sale prices rep.
# like: Internal family sale transfer at $0, garage sale etc.

ny18_df = ny18_df[ny18_df['SALE PRICE'].astype(float) > 100000]
ny18_df = ny18_df.drop_duplicates(subset=['ADDRESS'], keep = 'first')

In [10]:
len(ny18_df)

8951

In [ ]:
ny18_df.head()

In [ ]:
response = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address=635+west+42+street&citystatezip=10036&rentzestimate=true")

In [ ]:
print(response)

In [ ]:
#Using BeautifulSoup to convert XML text to pretty format
soup = BeautifulSoup(response.text)
print(soup.prettify())

In [ ]:
doc = xmltodict.parse(response.text)['SearchResults:searchresults']['response']['results']['result']
len(doc)

In [ ]:
doc[1]['address']['street']

In [ ]:
def need_rent_estimate(address):
    return (address.get('rentzestimate')) and not(address.get('lastSoldPrice'))

In [ ]:
def need_sale_estimate(address):
    return (not(address.get('rentzestimate')) and (address.get('lastSoldPrice')))

In [ ]:
def get_rent_estimate(address):
    print("Entering get_rent_estimate")
    rent = address['rentzestimate']['amount']['#text']
    address = address['address']['street']
#     zipcode = address['address']['zipcode']
    var = 2
#     return pd.Series({'rent':rent, 'address':address,'zipcode':zipcode,'var':var})
    return pd.Series({'rent':rent, 'address':address,'var':var})

In [ ]:
def get_property_estimate(address):
    print("Entering get_property_estimate")
    sold = address['lastSoldPrice']['#text']
    sold_date = address['lastSoldDate']
#     zestimate=address['zestimate']['amount']['#text']
    sqft = address['finishedSqFt']
    address = address['address']['street']
#     zipcode = address['address']['zipcode']
    var = 1
    return pd.Series({'sold':sold, 'var':var}) 

In [ ]:
ny18_df = ny18_df.reset_index()
ny18_df.head()

In [ ]:
ny18_df.loc[0,'BLOCK']

In [ ]:
block =[]
lot=[]
ny_sale_amt =[]
ny_sale_date=[]
series = []
curdoc={}

for address, zipcode in zip(ny18_df['ADDRESS'],ny18_df['ZIP CODE']):
    try:
       i = 0
       print(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={address}&citystatezip={zipcode}&rentzestimate=true")
       response_main = requests.get(f"https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id={z_keys}&address={address}&citystatezip={zipcode}&rentzestimate=true")
#        block=ny18_Update['BLOCK']
#        lot=ny18_Update['LOT']
#        sale_amt=ny18_Update['SALE PRICE']
#        sale_date=ny18_Update['SALE DATE']
       doc = xmltodict.parse(response_main.text, force_list={'result': True})['SearchResults:searchresults']['response']['results']['result']

       while i < len(doc):
           print(f"{doc[i]['address']['street']} \n")

           if need_rent_estimate(doc[i]):
               new_series = get_rent_estimate(doc[i])
           elif need_sale_estimate(doc[i]):
               new_series = get_property_estimate(doc[i])
           
           i+=1
           print(f'End of record {i}')
           print('-------------------------------')
           series.append(new_series)
       time.sleep(30)
       df = pd.concat(series, 1,sort=True).T
    except KeyError:
       print(f"Only 1 record: {doc[i]['address']['street']} \n")

In [ ]:
# xmltodict.parse(response_main.text)['SearchResults:searchresults'].keys()